In [1]:
import sys
import os
import pandas as pd
import spacy
import ast

%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(PATH)

In [286]:
data = pd.read_csv(PATH + '/skills_matcher/data/data_final_raw.csv')
data.position.nunique()

857

In [287]:
new_data = data.drop_duplicates(subset ="position").reset_index()
new_data.to_csv(PATH + '/skills_matcher/data/data_final_raw_new.csv')

In [288]:
new_data

,index,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label
0,0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'..."
1,3,3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...,"(Python, Python)","[{'entity': Python, 'label': 'SKILL'}, {'entit..."
2,6,6,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...,"(medium,)","[{'entity': rel=""nofollow"">http://madx.web.cer..."
3,9,9,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...,(),"[{'entity': Good communication, 'label': 'SKIL..."
4,11,11,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Graduate Experimental Engineering Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...,(),"[{'entity': Top Rates, 'label': 'SKILL'}]"
...,...,...,...,...,...,...,...,...,...,...
852,3475,3475,22,HEALTH PROFESSIONALS,group leader,Customer Technical Services (CTS) Group Leader,Berkshire,"["" Drive delivery of the regional Sales Plan t...",(),"[{'entity': 3-5 years, 'label': 'MIN_EXP'}]"
853,3476,3476,22,HEALTH PROFESSIONALS,group leader,Development Group Leader,England,[' Demonstrate leadership in EHS by ensuring c...,(),"[{'entity': branded T-Shirts, 'label': 'KNOWLE..."
854,3477,3477,22,HEALTH PROFESSIONALS,group leader,Group Leader Molecular Engineering,Zurich,[' Design and optimization of protein sequence...,"(fusion, fusion)",[]
855,3478,3478,22,HEALTH PROFESSIONALS,group leader,Group Researcher to the Opposition / Group Leader,"Oldham, Greater Manchester","[' To collect and analyse data, internal and e...",(),"[{'entity': Expedia, 'label': 'SKILL'}]"


In [64]:
def extract_skills_auto(s):
    words_pattern = '[a-z]+'
    return re.findall(words_pattern, s, flags=re.IGNORECASE)

In [129]:
import re

#  new with non word char inside entities
def extract_entities_2(s):
    pattern = re.compile("'entity': [a-zA-Z]+\W*([a-zA-Z]|\s)*")
    res = re.finditer(pattern, s)
    return [ mo.group().split(",")[0].strip().lstrip("'entity': ") for mo in res if mo.group() != ',']

def extract_lables(s):
    pattern = re.compile("'label':\s*'\w+'")
    res = re.finditer(pattern, s)
    return [ mo.group().partition("': '")[2].strip("'") for mo in res]

In [159]:
def get_dict(x):
    
    dict_ = {}
    
    entities = extract_entities_2(x['entities_manual_label'])
    labels = extract_lables(x['entities_manual_label'])
    for index, key in enumerate(entities):
        if key not in dict_.keys():
            dict_[key] = labels[index]

    skills_auto =extract_skills_auto(x['entities_auto_label'])
    for index, value in enumerate(skills_auto):
        if value not in dict_.keys():
            dict_[value] = 'SKILL'
    
    skills_list = []
    knowledge_list = []
    min_exp_list = []
    level_list = []
            
    for entity, label in dict_.items():
        if label == 'SKILL':
            skills_list.append(entity)
        if label == 'KNOWLEDGE':
            knowledge_list.append(entity)
        if label == 'MIN_EXP':
             min_exp_list.append(entity)
        if label == 'LEVEL':
            level_list.append(entity)
    
    x["SKILL"] = skills_list
    x["KNOWLEDGE"] = knowledge_list
    x["MIN_EXP"] = min_exp_list
    x["LEVEL"] = level_list
    
    return x

In [289]:
data_cleaned = new_data.apply(get_dict, axis=1)

In [290]:
data_cleaned

,index,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label,SKILL,KNOWLEDGE,MIN_EXP,LEVEL
0,0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'...","[English, German]",[physics],[],[]
1,3,3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...,"(Python, Python)","[{'entity': Python, 'label': 'SKILL'}, {'entit...","[Python, English, French]","[programming, High Energy Physics data analysi...",[],[]
2,6,6,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...,"(medium,)","[{'entity': rel=""nofollow"">http://madx.web.cer...",[medium],"[rel=""nofollow]",[],[]
3,9,9,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...,(),"[{'entity': Good communication, 'label': 'SKIL...",[Good communication],[],[],[]
4,11,11,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Graduate Experimental Engineering Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...,(),"[{'entity': Top Rates, 'label': 'SKILL'}]",[Top Rates],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,3475,3475,22,HEALTH PROFESSIONALS,group leader,Customer Technical Services (CTS) Group Leader,Berkshire,"["" Drive delivery of the regional Sales Plan t...",(),"[{'entity': 3-5 years, 'label': 'MIN_EXP'}]",[],[],[],[]
853,3476,3476,22,HEALTH PROFESSIONALS,group leader,Development Group Leader,England,[' Demonstrate leadership in EHS by ensuring c...,(),"[{'entity': branded T-Shirts, 'label': 'KNOWLE...",[],[branded T],[],[]
854,3477,3477,22,HEALTH PROFESSIONALS,group leader,Group Leader Molecular Engineering,Zurich,[' Design and optimization of protein sequence...,"(fusion, fusion)",[],[fusion],[],[],[]
855,3478,3478,22,HEALTH PROFESSIONALS,group leader,Group Researcher to the Opposition / Group Leader,"Oldham, Greater Manchester","[' To collect and analyse data, internal and e...",(),"[{'entity': Expedia, 'label': 'SKILL'}]",[Expedia],[],[],[]


In [147]:
data_cleaned['SKILL'][0]

['English', 'German']

In [297]:
data_cleaned.index[data_cleaned['position'] == position].tolist()

[0]

In [303]:
def print_final_table(position):
    
    index = data_cleaned.index[data_cleaned['position'] == position].to_list()
    
    list_skills = data_cleaned['SKILL'][index]
    list_knowledge = data_cleaned['KNOWLEDGE'][index]
    list_min_exp = data_cleaned['MIN_EXP'][index]
    list_level = data_cleaned['LEVEL'][index]
    
    return pd.DataFrame({'skills': pd.Series(list_skills), 'knowledge': pd.Series(list_knowledge)})

In [305]:
print_final_table('Customer Technical Services (CTS) Group Leader')

,skills,knowledge
852,[],[]


In [307]:
def create_sum(s):
    number = len(s['SKILL']) + len(s['KNOWLEDGE']) + len(s['MIN_EXP']) + len(s['LEVEL'])
    s['sum'] = number
    return s

In [308]:
data_cleaned.iloc[0]['SKILL']

['English', 'German']

In [310]:
sum_column = data_cleaned['SKILL'] + data_cleaned['KNOWLEDGE'] + data_cleaned['MIN_EXP'] + data_cleaned['LEVEL']
data_cleaned['SUM'] = sum_column

In [311]:
sums = []
for index, row in enumerate(data_cleaned['SUM']):
    sums.append(len(data_cleaned['SUM'][index]))
    
data_cleaned['SUM_num'] = sums

In [312]:
for skill in data_cleaned['SKILL'][1]:
    print(skill.capitalize())

Python
English
French


In [313]:
data_cleaned['LEVEL'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                                  518
[Degree]                                             38
[PhD]                                                35
[BSc]                                                15
[PCV]                                                11
                                                   ... 
[Toxicology]                                          1
[AutoCAD]                                             1
[MSc and PhD]                                         1
[Degree, Three]                                       1
[few years sole charge experience Accommodation]      1
Name: LEVEL, Length: 123, dtype: int64

In [314]:
data_cleaned['SUM_num'].value_counts()

1     162
3     134
2     132
4      77
5      59
6      59
7      46
9      39
8      36
0      23
11     21
12     17
10     13
13      9
15      6
14      6
18      3
17      3
21      2
27      2
28      2
19      2
20      1
29      1
26      1
16      1
Name: SUM_num, dtype: int64